In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
%cd /content/drive/MyDrive/Colab Notebooks/interaction-llm-research

/content/drive/MyDrive/Colab Notebooks/interaction-llm-research


In [38]:
import pandas as pd
import geopandas as gpd

# Import data

In [39]:
# import taxi trip data
file_path = 'data/yellow_tripdata_2025-07.parquet'

try:
    df = pd.read_parquet(file_path)
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-07-01 00:29:37,2025-07-01 00:45:30,1.0,7.30,1.0,N,138,74,1,29.6,7.75,0.5,9.00,6.94,1.0,54.79,0.0,1.75,0.00
1,1,2025-07-01 00:23:28,2025-07-01 01:07:44,1.0,17.70,2.0,N,132,142,1,70.0,4.25,0.5,5.00,0.00,1.0,80.75,2.5,1.75,0.00
2,2,2025-07-01 00:53:50,2025-07-01 01:27:12,1.0,9.98,1.0,N,138,48,1,43.6,6.00,0.5,10.87,0.00,1.0,66.97,2.5,1.75,0.75
3,2,2025-07-01 00:58:49,2025-07-01 01:15:55,1.0,10.27,1.0,N,138,229,1,38.7,6.00,0.5,14.10,6.94,1.0,72.24,2.5,1.75,0.75
4,2,2025-07-01 00:09:22,2025-07-01 00:23:54,1.0,2.94,1.0,N,211,97,1,17.0,1.00,0.5,3.00,0.00,1.0,25.75,2.5,0.00,0.75


In [40]:
# Check the variables in the DataFrame
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3898963 entries, 0 to 3898962
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

None

In [41]:
# Filter the DataFrame to keep only the specified columns
df = df[['tpep_pickup_datetime', 'PULocationID', 'DOLocationID']]

# Display the first few rows of the modified DataFrame to verify
display(df.head())

,tpep_pickup_datetime,PULocationID,DOLocationID
0,2025-07-01 00:29:37,138,74
1,2025-07-01 00:23:28,132,142
2,2025-07-01 00:53:50,138,48
3,2025-07-01 00:58:49,138,229
4,2025-07-01 00:09:22,211,97


In [42]:
# Check the number of observations (rows)
num_observations = len(df)
print(f"Number of observations (rows): {num_observations}")

# Check the number of missing values per column
missing_values = df.isnull().sum()
print("\nNumber of missing values per column:")
print(missing_values)

# Check the total number of missing values in the DataFrame
total_missing_values = missing_values.sum()
print(f"\nTotal number of missing values in the DataFrame: {total_missing_values}")

Number of observations (rows): 3898963

Number of missing values per column:
tpep_pickup_datetime    0
PULocationID            0
DOLocationID            0
dtype: int64

Total number of missing values in the DataFrame: 0


In [43]:
# import taxi zone shapefile
gdf = gpd.read_file('data/taxi_zones.shp')

In [44]:
display(gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    263 non-null    int32   
 1   Shape_Leng  263 non-null    float64 
 2   Shape_Area  263 non-null    float64 
 3   zone        263 non-null    object  
 4   LocationID  263 non-null    int32   
 5   borough     263 non-null    object  
 6   geometry    263 non-null    geometry
dtypes: float64(2), geometry(1), int32(2), object(2)
memory usage: 12.5+ KB


None

In [45]:
# Filter gdf to get rows where borough is 'Manhattan'
manhattan_zones = gdf[gdf['borough'] == 'Manhattan']

# Get the list of LocationIDs for Manhattan
manhattan_location_ids = manhattan_zones['LocationID'].tolist()

# Filter df to keep trips where both PULocationID and DOLocationID are in Manhattan
manhattan_trips_df = df[
    (df['PULocationID'].isin(manhattan_location_ids)) &
    (df['DOLocationID'].isin(manhattan_location_ids))
]

# Display the first few rows of the filtered DataFrame
display(manhattan_trips_df.head())

# Print the number of trips within Manhattan
print(f"\nNumber of trips within Manhattan: {len(manhattan_trips_df)}")

,tpep_pickup_datetime,PULocationID,DOLocationID
6,2025-07-01 00:15:26,79,263
7,2025-07-01 00:40:58,140,262
8,2025-07-01 00:28:12,114,50
10,2025-07-01 00:44:56,233,233
14,2025-07-01 00:02:08,229,163



Number of trips within Manhattan: 3048178


In [46]:
# Filter out trips where pickup and dropoff locations are the same
filtered_manhattan_trips_df = manhattan_trips_df[manhattan_trips_df['PULocationID'] != manhattan_trips_df['DOLocationID']]

# Group by pickup and dropoff locations and count the number of trips
flow = filtered_manhattan_trips_df.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='trip_count')

# Display the first few rows of the flow DataFrame
display(flow.head())

# Print the total number of unique origin-destination pairs
print(f"\nTotal number of unique origin-destination pairs: {len(flow)}")

,PULocationID,DOLocationID,trip_count
0,4,12,1
1,4,13,106
2,4,24,31
3,4,41,67
4,4,42,133



Total number of unique origin-destination pairs: 4113


# Make Binary Contiguity Matrix and Distance-weighted Matrix(Edge)



In [47]:
import numpy as np
from shapely.geometry import Point
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import libpysal as lps
import geopandas as gpd

# Ensure gdf is available (assuming it was loaded in a previous cell)
try:
    # Filter gdf to get rows where borough is 'Manhattan'
    manhattan_zones = gdf[gdf['borough'] == 'Manhattan'].copy() # Use .copy() to avoid SettingWithCopyWarning
    if manhattan_zones.empty:
        print("Error: No zones found for Manhattan.")
except NameError:
    print("Error: gdf GeoDataFrame not found. Please ensure the cell loading gdf is executed.")
    manhattan_zones = gpd.GeoDataFrame() # Create an empty GeoDataFrame to prevent further errors
except Exception as e:
    print(f"An error occurred while filtering Manhattan zones: {e}")
    manhattan_zones = gpd.GeoDataFrame()

# 1. Prepare spatial data
if not manhattan_zones.empty:
    # Ensure the index of manhattan_zones is set to LocationID for easier lookup
    if 'LocationID' in manhattan_zones.columns:
        manhattan_zones = manhattan_zones.set_index('LocationID')
    else:
        print("Error: 'LocationID' column not found in manhattan_zones.")
        manhattan_zones = gpd.GeoDataFrame() # Reset to empty if essential column is missing

if not manhattan_zones.empty:
    # Sort the index to ensure consistent ordering for matrix creation
    manhattan_zones = manhattan_zones.sort_index()

    # Get the list of LocationIDs in the sorted order
    location_ids_sorted = manhattan_zones.index.tolist()
    num_zones = len(location_ids_sorted)

    # Create a mapping from LocationID to index in the sorted list
    location_id_to_index = {loc_id: index for index, loc_id in enumerate(location_ids_sorted)}

    # Print some info for debugging
    print(f"Number of Manhattan zones: {num_zones}")


    # 2. Calculate centroids (Needed for finding nearest neighbors of islands)
    try:
        # Calculate the centroid for each zone's geometry
        manhattan_zones['centroid'] = manhattan_zones['geometry'].centroid
        centroids = np.array([[p.x, p.y] for p in manhattan_zones['centroid']])
        print(f"Shape of centroids array: {centroids.shape}")

    except Exception as e:
        print(f"An error occurred while calculating centroids: {e}")
        centroids = np.array([]) # Ensure centroids is an empty array if calculation fails


    # 3. Create initial adjacency matrix (using Queen contiguity)
    adjacency_matrix = np.zeros((num_zones, num_zones)) # Initialize with zeros
    islands = [] # List to store island LocationIDs

    if centroids.shape[0] == num_zones: # Proceed only if centroids were calculated correctly
        try:
            print("Attempting to create Queen contiguity weights...")
            # Create Queen contiguity weights (zones sharing a border or a vertex are considered adjacent)
            # Using ids=manhattan_zones.index to ensure correct mapping
            wq = lps.weights.Queen.from_dataframe(manhattan_zones, ids=manhattan_zones.index)
            print("Queen contiguity weights created.")

            # Convert the weights object to a sparse matrix and then to a dense numpy array
            # Ensure the matrix is ordered according to the sorted index
            adjacency_matrix_wq = wq.sparse.toarray()

            # Get the list of islands identified by libpysal
            islands = wq.islands
            print(f"Islands found by Queen contiguity: {islands}")

            # Copy the initial contiguity matrix
            adjacency_matrix = adjacency_matrix_wq.copy()


        except ImportError:
            print("Error: libpysal is required for calculating spatial adjacency.")
            print("Please install it using: !pip install libpysal")
            print("A placeholder zero matrix is created.")
        except Exception as e:
            print(f"An error occurred during Queen contiguity matrix creation: {e}")
            print("Proceeding to handle islands with a zero matrix if libpysal failed.")
            # If libpysal failed, islands list might be empty, need to re-identify if possible or skip island handling


        # Handle islands: connect each island to its nearest neighbors (4 neighbors)
        if islands and centroids.shape[0] == num_zones: # Proceed only if islands found and centroids are correct
            print(f"Handling {len(islands)} islands...")
            k_neighbors = 4 # Number of nearest neighbors to connect

            # Use k-NN to find the nearest neighbors for each island
            # Exclude islands from the training data for k-NN to find neighbors *not* among islands
            non_island_indices = [i for i in range(num_zones) if location_ids_sorted[i] not in islands]

            if non_island_indices: # Check if there are non-island zones to connect to
                non_island_centroids = centroids[non_island_indices]
                island_centroids = centroids[[location_id_to_index[loc_id] for loc_id in islands]]

                if non_island_centroids.shape[0] >= k_neighbors: # Ensure enough non-island neighbors exist
                    print(f"Finding {k_neighbors} nearest neighbors for {len(islands)} islands among {non_island_centroids.shape[0]} non-island zones...")
                    nn = NearestNeighbors(n_neighbors=k_neighbors, algorithm='ball_tree')
                    nn.fit(non_island_centroids)

                    distances, indices = nn.kneighbors(island_centroids)

                    # Add connections in the adjacency matrix
                    for i, island_loc_id in enumerate(islands):
                        island_index = location_id_to_index[island_loc_id]
                        nearest_neighbor_indices_in_non_islands = indices[i]

                        for nearest_neighbor_index_in_non_islands in nearest_neighbor_indices_in_non_islands:
                            nearest_neighbor_loc_id = location_ids_sorted[non_island_indices[nearest_neighbor_index_in_non_islands]]
                            nearest_neighbor_index = location_id_to_index[nearest_neighbor_loc_id]

                            # Add edge in both directions (undirected graph)
                            adjacency_matrix[island_index, nearest_neighbor_index] = 1
                            adjacency_matrix[nearest_neighbor_index, island_index] = 1
                            # print(f"Connected island {island_loc_id} (index {island_index}) to nearest neighbor {nearest_neighbor_loc_id} (index {nearest_neighbor_index}).") # Uncomment for detailed connection info

                else:
                     print(f"Not enough non-island zones ({non_island_centroids.shape[0]}) to connect {k_neighbors} neighbors to islands.")
            else:
                 print("No non-island zones available to connect islands to.")
        elif islands:
             print("Skipping island handling due to centroid calculation error.")
        else:
             print("No islands found, island handling skipped.")


        # Ensure the diagonal is 0 (a zone is not adjacent to itself)
        np.fill_diagonal(adjacency_matrix, 0)

        # Display the shape of the binary adjacency matrix
        print(f"Shape of the final binary adjacency matrix: {adjacency_matrix.shape}")
        # display(adjacency_matrix[:5, :5]) # Uncomment to display a portion of the matrix
    else:
        print("Skipping matrix creation due to centroid calculation error or empty manhattan_zones.")
else:
    print("manhattan_zones is empty, skipping matrix creation.")

Number of Manhattan zones: 69
Shape of centroids array: (69, 2)
Attempting to create Queen contiguity weights...
An error occurred during Queen contiguity matrix creation: The argument to the ids parameter contains duplicate entries.
Proceeding to handle islands with a zero matrix if libpysal failed.
No islands found, island handling skipped.
Shape of the final binary adjacency matrix: (69, 69)


In [48]:
import numpy as np
from scipy.spatial.distance import cdist

# 3. Calculate centroids
# Calculate the centroid for each zone's geometry
manhattan_zones['centroid'] = manhattan_zones['geometry'].centroid

# Extract centroid coordinates
centroids = np.array([[p.x, p.y] for p in manhattan_zones['centroid']])

# 4. Calculate distance matrix
# Calculate pairwise Euclidean distances between centroids
try:
    distance_matrix = cdist(centroids, centroids)

    # Display the shape of the distance matrix
    print(f"Shape of the distance matrix: {distance_matrix.shape}")
    # display(distance_matrix[:5, :5]) # Uncomment to display a portion of the matrix

    # 5. Create distance-weighted matrix (using Gaussian kernel)
    # Define the Gaussian kernel function as used in the specified paper
    def gaussian_kernel(distance, sigma):
        return np.exp(-distance**2 / (sigma**2))

    # Choose a suitable sigma value (this may require tuning)
    # A common approach is to use the average distance or a fraction of the max distance
    # For now, let's use a simple heuristic, e.g., the mean distance, avoiding zero distances
    # Check if there are non-zero distances to calculate the mean
    non_zero_distances = distance_matrix[distance_matrix > 0]
    if non_zero_distances.size > 0:
        sigma = np.mean(non_zero_distances)
    else:
        # Handle the case where all distances are zero (e.g., only one zone)
        sigma = 1.0 # Or another appropriate default/error handling

    # Apply the Gaussian kernel to the distance matrix
    # Set the diagonal to 0 as a zone is not 'distant' from itself in this context
    distance_weighted_matrix = gaussian_kernel(distance_matrix, sigma)
    np.fill_diagonal(distance_weighted_matrix, 0)

    # Display the shape of the distance-weighted matrix
    print(f"Shape of the distance-weighted matrix: {distance_weighted_matrix.shape}")
    # display(distance_weighted_matrix[:5, :5]) # Uncomment to display a portion of the matrix

    print("\nBinary adjacency matrix ('adjacency_matrix') and distance-weighted matrix ('distance_weighted_matrix') have been created.")
    print("These matrices can now be used for your graph deep learning model.")

except NameError:
    print("Error: 'centroids' variable not found. Please ensure the previous cell calculating centroids was executed successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Shape of the distance matrix: (69, 69)
Shape of the distance-weighted matrix: (69, 69)

Binary adjacency matrix ('adjacency_matrix') and distance-weighted matrix ('distance_weighted_matrix') have been created.
These matrices can now be used for your graph deep learning model.


# Add Node Features
